In [ ]:
!pip install transformers datasets scikit-learn torch --quiet

In [ ]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# --- Load dataset ---
df = pd.read_csv('hate_sample_for_finetune.csv')
df = df.dropna(subset=['hate_type_teacher'])

# --- Balance classes ---
df_majority = df[df['hate_type_teacher'] == 'sarcasm-based hate']
df_minority = df[df['hate_type_teacher'] != 'sarcasm-based hate']

# Oversample minority classes to match majority count
df_minority_upsampled = resample(
    df_minority,
    replace=True,
    n_samples=len(df_majority),
    random_state=42
)


In [ ]:
# Combine and shuffle
df_balanced = pd.concat([df_majority, df_minority_upsampled])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print(df_balanced['hate_type_teacher'].value_counts())

hate_type_teacher
sarcasm-based hate        1267
metaphor-based hate        530
meme-language hate         321
humor-based hate           221
political framing hate     195
Name: count, dtype: int64


In [ ]:
# --- Encode labels ---
label_mapping = {label: idx for idx, label in enumerate(df_balanced['hate_type_teacher'].unique())}
df_balanced['hate_type_teacher_int'] = df_balanced['hate_type_teacher'].map(label_mapping)
print("Label mapping:", label_mapping)

Label mapping: {'metaphor-based hate': 0, 'sarcasm-based hate': 1, 'humor-based hate': 2, 'meme-language hate': 3, 'political framing hate': 4}


In [ ]:
# --- Split into train and validation ---
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_balanced['clean_text'],
    df_balanced['hate_type_teacher_int'],
    test_size=0.2,
    random_state=42,
    stratify=df_balanced['hate_type_teacher_int']
)

In [ ]:
# --- Tokenization ---
model_name = "roberta-base"  # change here

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_mapping))

# Tokenize train and validation data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=256)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# --- Custom Dataset ---
class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.reset_index(drop=True)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(int(self.labels.iloc[idx]), dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HateSpeechDataset(train_encodings, train_labels)
val_dataset = HateSpeechDataset(val_encodings, val_labels)


In [ ]:
# --- Metrics Function ---
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = preds.argmax(axis=1)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': prec, 'recall': rec}


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    logging_strategy="epoch",
    report_to=[],
    learning_rate=2e-5
)

In [ ]:
# --- Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


In [ ]:
# --- Train & Evaluate ---
trainer.train()
metrics = trainer.evaluate()
print(metrics)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.323400,1.238408,0.500986,0.334430,0.250987,0.500986
2,1.196900,1.186891,0.504931,0.455170,0.452006,0.504931
3,1.103000,1.128517,0.540434,0.509061,0.503632,0.540434


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 1.1285172700881958, 'eval_accuracy': 0.5404339250493096, 'eval_f1': 0.5090607536121615, 'eval_precision': 0.5036321971249231, 'eval_recall': 0.5404339250493096, 'eval_runtime': 27.989, 'eval_samples_per_second': 18.114, 'eval_steps_per_second': 1.143, 'epoch': 3.0}


In [ ]:
# --- Save Model ---
model.save_pretrained('./distilbert-hate-balanced')
tokenizer.save_pretrained('./distilbert-hate-balanced')

('./distilbert-hate-balanced/tokenizer_config.json',
 './distilbert-hate-balanced/special_tokens_map.json',
 './distilbert-hate-balanced/vocab.json',
 './distilbert-hate-balanced/merges.txt',
 './distilbert-hate-balanced/added_tokens.json',
 './distilbert-hate-balanced/tokenizer.json')

In [ ]:
# --- Classification Report ---
from sklearn.metrics import classification_report
predictions = trainer.predict(val_dataset)
pred_labels = predictions.predictions.argmax(axis=1)
true_labels = predictions.label_ids
print("Classification Report:\n", classification_report(true_labels, pred_labels))

Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.48      0.51       106
           1       0.71      0.81      0.75       254
           2       0.00      0.00      0.00        44
           3       0.46      0.59      0.52        64
           4       0.38      0.38      0.38        39

    accuracy                           0.61       507
   macro avg       0.42      0.45      0.43       507
weighted avg       0.56      0.61      0.58       507



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
